In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

True

In [ ]:
ALL_IN_ONE_SG = 'sg-0d8a868137f653df6'
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
AWS_INSTANCE_ID_JMASTER   = os.environ['AWS_INSTANCE_ID_JMASTER']
AWS_INSTANCE_ID_JAGENT    = os.environ['AWS_INSTANCE_ID_JAGENT']
AWS_DEFAULT_IMAGE_ID      = os.environ['AWS_DEFAULT_IMAGE_ID']
AWS_DEFAULT_KEY_PAIR_NAME = os.environ['AWS_DEFAULT_KEY_PAIR_NAME']
AWS_DEFAULT_INSTANCE_TYPE = os.environ['AWS_DEFAULT_INSTANCE_TYPE']

In [ ]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')
sqs_client           = boto3.client('sqs')

emr_client = boto3.client('emr', region_name=REGION)

In [ ]:
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)

# # Example: Get a specific VPC
# vpc = ec2_resource.Vpc('vpc_id')

# # Example: Get a specific EBS volume
# volume = ec2_resource.Volume('volume_id')

### [DNS & Amazon Route 53 Deep dive](https://www.youtube.com/watch?v=94vdYMBcE5Y&list=PLO95rE9ahzRsP_ryXpnAHTbKlQIt_JEyF&index=1)

In [1]:
! mkdir ./data/route53

In [2]:
! curl -o ./data/route53/free_static_webpage.zip https://www.free-css.com/assets/files/free-css-templates/download/page295/makaan.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  877k  100  877k    0     0   468k      0  0:00:01  0:00:01 --:--:--  470k


In [ ]:
! unzip ./data/route53/free_static_webpage.zip -d ./data/route53/

In [8]:
! rm -fr ./data/route53/free_static_webpage.zip

In [ ]:
def create_dns_record(
    zone_id,
    record_name,
    record_type,
    record_value,
    ttl=300,
    routing_policy="simple",
    set_identifier=None,
    weight=None,
    region=None,
):
    """
    Creates a DNS record in AWS Route 53 with the specified routing policy.

    :param zone_id: (str) Hosted zone ID.
    :param record_name: (str) Fully qualified domain name (FQDN).
    :param record_type: (str) DNS record type (e.g., "A", "CNAME", "TXT").
    :param record_value: (list) List of values for the record.
    :param ttl: (int) Time to Live (TTL) in seconds.
    :param routing_policy: (str) Routing policy ("simple", "weighted", "latency", "failover", "geolocation").
    :param set_identifier: (str) Unique identifier required for weighted, latency, geolocation policies.
    :param weight: (int) Weight value (required for weighted policy).
    :param region: (str) AWS region (required for latency policy).
    """
    client = boto3.client("route53")

    record_set = {
        "Name": record_name,
        "Type": record_type,
        "TTL": ttl,
        "ResourceRecords": [{"Value": val} for val in record_value],
    }

    # Handle routing policies
    if routing_policy == "weighted":
        if set_identifier is None or weight is None:
            raise ValueError("Weighted routing requires 'set_identifier' and 'weight'")
        record_set["SetIdentifier"] = set_identifier
        record_set["Weight"] = weight

    elif routing_policy == "latency":
        if set_identifier is None or region is None:
            raise ValueError("Latency routing requires 'set_identifier' and 'region'")
        record_set["SetIdentifier"] = set_identifier
        record_set["Region"] = region

    elif routing_policy == "failover":
        if set_identifier is None:
            raise ValueError("Failover routing requires 'set_identifier'")
        record_set["SetIdentifier"] = set_identifier
        record_set["Failover"] = "PRIMARY"  # Change to "SECONDARY" as needed

    elif routing_policy == "geolocation":
        if set_identifier is None:
            raise ValueError("Geolocation routing requires 'set_identifier'")
        record_set["SetIdentifier"] = set_identifier
        record_set["GeoLocation"] = {"CountryCode": "US"}  # Modify as needed

    change_batch = {"Changes": [{"Action": "UPSERT", "ResourceRecordSet": record_set}]}

    response = client.change_resource_record_sets(
        HostedZoneId=zone_id, ChangeBatch=change_batch
    )

    return response

In [ ]:
# # Example Usage:
# response = create_dns_record(
#     zone_id="ZXXXXXXXXXXXXX",
#     record_name="example.mydomain.com",
#     record_type="A",
#     record_value=["192.168.1.1"],
#     ttl=300,
#     routing_policy="weighted",
#     set_identifier="Instance-1",
#     weight=50,
# )
# print(response)